<a href="https://colab.research.google.com/github/arpdm/apcs/blob/main/apcs_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pathlib
import tarfile
import urllib.request

In [3]:
core_path = ""

# Set the path for downloading data files
pathlib.Path(os.path.join(core_path, "data/smass_data/")).mkdir(parents=True, exist_ok=True)